In [2]:
import glob
import os
import string
import re
from scipy.sparse import dok_matrix
import numpy as np

## The goal of this notebook is to learn Markov chains with a real application to the Malazan Book Series

In [3]:
files = glob.glob("C:/Users/aiden/Dropbox/BOOKS/Malazan Series/SE/Steven Erikson - Malazan Book of the Fallen 10 Books/txt/*")
files = [f for f in files if 'train' in f]

To begin, we read in pre-processed text files of the book, stripping out new lines, and treating punctuation as unique word tokens for prediction (by surrounding each punctuation with a space).

In [4]:
puncts_processed = '|'.join([re.escape(s) for s in string.punctuation])

def read_text(f):
    ### Purpose of this function is to read in text and do some basic cleaning to produce a list of words
    ###  Punctuation are replaced by surrounded by space to incorporate them into the model as distinct tokens
    lines = open(f, 'r').readlines()
    lines = [re.sub("\n|\t|\r", " ", l) for l in lines]
    lines = [re.sub(rf'({puncts_processed})', r" \1 ", l) for l in lines]
    return ' '.join(lines)

In [5]:
## Read in all books
book = ' '.join([read_text(f) for f in files])

In [6]:
corpus_words = [w for w in book.split(" ") if w != '']

In [7]:
unq_words = set(corpus_words)
print(f"There are {len(unq_words)} unique words in the Malazan Book of the Fallen Series")

There are 32775 unique words in the Malazan Book of the Fallen Series


Now that we have read in all 10 books and combined them into one long list of sequential words, we need to prepare the transition matrix.  The first step is to generate sequences of `i`, `i + 1` words where we are predicting the probability of the next word based only on  the previous one word.

In [ ]:
## Generate transition matrix
k = 1 # adjustable
word_sequences = [ ' '.join(corpus_words[i:i+k]) for i, _ in enumerate(corpus_words[:-k]) ]

In [13]:
sets_count = len(list(set(word_sequences)))
transition_matrix = dok_matrix((sets_count, len(unq_words)))
distinct_word_seq = list(set(word_sequences))
# Construct word to idx and idx to word dictionaries
word_idx_dict = {word: i for i, word in enumerate(distinct_word_seq)}
idx_word_dict = {i: word for i, word in enumerate(distinct_word_seq)}

for i, word in enumerate(word_sequences[:-k]):

    word_sequence_idx = k_words_idx_dict[word]
    next_word_idx = word_idx_dict[corpus_words[i+k]]
    transition_matrix[word_sequence_idx, next_word_idx] +=1

In [14]:
## Build out transition probability matrix
prob_matrix = transition_matrix.copy()
prob_matrix = prob_matrix / prob_matrix.sum(axis=1)

Great!  We built our transition matrix, let's sanity check this by seeing if rake follows anomander at a high probability.  "Anomander Rake" is a character in the series and so it should follow at a high probability

In [72]:
## Let's say, for Anomander Rake, what re see?
print(word_idx_dict['anomander']) ## word idx is 10996
rake_idx = np.where(prob_matrix[26725]>0.5)[1][0]
prob_rake = prob_matrix[26725].reshape(-1,1)[rake_idx].tolist()
print(f"Predicting probability for '{idx_word_dict[rake_idx]}' following word 'anomander' with probability {prob_rake} ")

26725
Predicting probability for 'rake' following word 'anomander' with probability [[0.805019305019305]] 


In [21]:
## Next thing we need is to conduct weighted randomly sampling based on a given input word
def predict_next_word(word):
    word_idx = k_words_idx_dict[word]
    pred_idx = np.random.choice( prob_matrix[word_idx].shape[1], p = np.ravel(prob_matrix[word_idx].reshape(-1,1).flatten()))
    return idx_word_dict[pred_idx]

In [22]:
## Now we are almost ready!   Predict the next X words given an input word
def predict_sequence(seed_word, n = 50):
    to_pred = seed_word
    prediction = [seed_word]
    for i in range(n):
        pred = predict_next_word(to_pred)
        prediction.append(pred)
        to_pred = pred
    return ' '.join(prediction)

In [24]:
## How does text prediction look?
predict_sequence("anomander")

'anomander rake a world it there a hawker the coast permanently so desperate flight between them i appreciate perhaps no gods below trull sengar came a hoof and spent lifetimes of the defences of the king s dead or she twisted his beard tell me i mean i ain t use'